A note on particle filters.

The particle filter is based on sampling an appropriate state $x_{t+1}$ based on the distribution $q(x_{t+1}|x_t,y_{t+1})$ and obtaining the importance weights

$$w_{t+1} \propto w_{t} \; \frac{f(x_{t+1}|x_{y}) g(y_{t+1}|x_t)}{q(x_{t+1}|x_t,y_{t+1})}$$

The problem is, we typically don't know all of the distributions in the above equation.  We do know $g$, since this is the influence of the measurement.  But we generally need to find some approximation for the distribution $q(x_{t+1}|x_t,y_{t+1})$.  A common choice is the so-called bootstrap filter, where we set

$$q(x_{t+1}|x_t,y_{t+1}) = f(x_{t+1}|x_{y})$$

which means that the weight function simplifies to

$$w_{t+1} \propto w_{t} \; g(y_{t+1}|x_t)$$

This procedure avoids having to actually construct the functional form for the distribution $q$, we only need to sample from it.  So in practice, we construct $x_{t+1}$ from $x_t$ by some stochastic process (eg integrating some Langevin equation), and then update the weights based on the measurement $y_{t+1}$.  This works reasonably well, but it isn't very efficient, because when we update the state to find $x_{t+1}$ we don't make any use of the (known) measurement $y_{t+1}$.

An alternative, is fully-adapted filtering, where we sample from $q$ itself, and we also need an analyical forms for $f$ and $q$.  How to find these?

$g$ is a known function -- this is just the probability of the gyro output, and the gyro has known variance. Indeed, $g(x_{t+1}|y_{t+1}) \sim N(\dot{x}_{t+1} - y_{t+1}, \sigma_g)$ where $\sigma_g$ is the noise of the gyro. Or $g(x_{t+1}|y_{t+1}) = \phi(x_{t+1};y_{t+1}, \sigma_g)$ where

$$\phi(x;m,\sigma) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left[ \frac{-(x-m)^2}{2\sigma^2} \right]$$

$f$ is the probability of state $x_{t+1}$ given state $x_t$.  We assume that the dynamics is done properly so that $\theta$ and $\dot{\theta}$ are exact and base the discussion on the (external) forces $F$.

A reasonable approximation would be to assume that $F$ is a random variable, such that the rate of change of $F$ isn't too big.  For the dynamics, $\dot{F} = -(g/l_b) \dot{\theta} \cos \theta$. Now $\cos \theta$ is a maximum at BDC and TDC. But the velocity $\dot{\theta}$ is a minimum near TDC, and a maximum near BDC.  So BDC is the region where the rate of change of force due to dynamical forces is greatest. This is also where we want the external force to be a minimum.

For $f(x_{t+1}|x_t)$ we can take $\Delta F \sim N(0, \sigma_f)$ where $\sigma_f$ is some distribution width, that can depend on $\theta$ and $\dot{\theta}$. We could also put a bound on $F_{t+1}$ itself, but we'll save that for later.  This form of $f$ is easy to sample from and also to evaluate.

Now we want a form for $q(x_{t+1}|x_t,y_{y+1})$.  Both $f$ and $g$ are of the form of gaussians. Given $y_{t+1}$ we can obtain a distribution for the expected force $F$, $(F_{t+1}|y_{t+1})$. This is essentially the change in velocity due to external forces that we need to correspond to having $\dot{x}_{t+1}$ agree with $y_{t+1}$.

The force that would make it agree exactly is

$$F^e_{t+1} = 2\frac{y_{t+1} - v_t}{\Delta t} - \dot{v}_t - F_t$$

where $\dot{v}_t$ is the dynamical forces, and assuming a mid-point rule. If we don't assume a mid-point rule, then we have

$$F^e_{t+1} = \frac{y_{t+1} - v_t}{\Delta t} - \dot{v}_t$$

Now $y_{t+1}$ is a random variable, so $F_{t+1}$ is as well, with a mean as above, and standard deviation of $\sigma_g / \Delta t$.

This is a normal distribution with mean $(g_{t+1} - v_{t})/\Delta t - \dot{v}_t$ and standard deviation $\sigma_g / \Delta t$.

We want to draw an $F_{t+1}$ based on $y_{t+1}$ and $x_{t}$, and also write down the distribution function. 

$$Pr(F_{t+1}=x) = \phi(x;F_t, \sigma_F/\Delta t) \phi(x; (g_{t+1}-v_t)/\Delta t - \dot{v}, \sigma_g / \Delta t)$$

The product of two normal distributions with means $m_1$ and $m_2$ and variances $\sigma^2_1$ and $\sigma^2_2$ is another normal distribution, with a prefactor.  The distribution mean is

$$m = \frac{m_1 \sigma^{-2}_1 + m_2 \sigma^{-2}_2}{\sigma^{-2}_1 + \sigma^{-2}_2}$$

and variance

$$\sigma^2 = \frac{\sigma^2_1 \sigma^2_2}{\sigma^2_1 + \sigma^2_2}$$

and the prefactor is

$$c = \phi(m_1; m_2, \sqrt{\sigma_1^2 + \sigma_2^2})$$

Note that we don't care about the prefactor when sampling, but we do care about it for evaluating $q$.

In our case, we have:

$$m = \frac{(g_{t+1}-v_t - \dot{v} \Delta t) \sigma_g^{-2}(\Delta t)}{\sigma_F^{-2} + \sigma_g^{-2} (\Delta t)^2}$$

$$\sigma^2 = \frac{\sigma_F^2 \sigma_g^2 / (\Delta t)^2}{\sigma_F^2 + \sigma_g^2/(\Delta t)^2}$$

$$c = \phi(0; (g_{t+1}-v_t)/\Delta t - \dot{v}_t, \sqrt{\sigma_F^2 + \sigma_g^2 / (\Delta t)^2})$$

Now we also need an explicit expression for $f(x_{t+1}|x_t)$. This is just $f(x_{t+1}|x_t) = \phi(F_{t+1}; F_t, \sigma_F / \Delta t)$.

This means that we have $w_{t+1} / w_t = f g / q$, or

$$w_{t+1} = w_t \; \phi(F_{t+1}; F_t, \sigma_F / \Delta t) \phi(x_{t+1};y_{t+1}, \sigma_g) / q$$

Does this just end up as $w_{t+1} = w_t$ ???

Yes, this is exactly the construction of the fully-adapted filter, all of the weights are identically equal to 1, and in principle every trajectory converges.

We can also consider a distribution on $F$ itself, since we don't expect that the forces themselves will be so big.  This should give better convergence, eg when the velocity is zero, we should converge to a stable fixed point with zero external force, but with no bound on $F$ trajectories could end up in long-lived states with a very large force.

If we take the product of 3 gaussian distributions, we end up with the obvious extension of the previous, another gaussian with some prefactor (not calculated here), and mean

$$m = \frac{m_1 \sigma_1^{-2} + m_2 \sigma_2^{-2} + m_3 \sigma_3^{-2}}{\sigma_1^{-2} + \sigma_2^{-2} + \sigma_3^{-2}}$$

and reciprocal variance

$$\sigma^{-2} = \sigma_1^{-2}+ \sigma_2^{-2} + \sigma_3^{-2}$$

Theta and velocity dependence on $\sigma_F$:

We can choose $\sigma_F$ based on $\theta$ and $\dot{\theta}$.  What is a good scheme?

We could also put some constraints on $F$ itself.  An obvious one is that the force on the rope always has the same sign.

In practice, it seems that trying to put restrictions on $\Delta F$ doesn't work so well.  With no restrictions, $F$ is very noisy, but some additional filtering, eg 200 point moving average, produces a reasonable result.  But this means that we can't prevent $\Delta F$ from being too big.

Putting a restriction on $\Delta F$ makes the system oscillate.  Basically, speed wobbles.  Maybe this will work if we can introduce some dampening.

But it also seeoms to work OK to allow $\Delta F$ to be big, and just smooth it later.